<a href="https://colab.research.google.com/github/thinothw/DFDS-Final-Project/blob/main/Phase01_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import pandas
import torch
print("✅ All core imports successful. The environment is stable.")

✅ All core imports successful. The environment is stable.


In [1]:
# Clean, compatible install block
!pip uninstall -y numpy -q
!pip install -q numpy==1.26.4
!pip install -q datasets==2.18.0
!pip install -q facenet-pytorch==2.5.3
!pip install -q opencv-python==4.8.0.76

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-pytho

In [3]:

# 1. Mount Google Drive

from google.colab import drive
import os
from datasets import load_dataset
import torch

print("Requesting Google Drive access...")
drive.mount('/content/drive')

# Verify Drive mounted correctly
if not os.path.exists('/content/drive/MyDrive'):
    raise RuntimeError("Google Drive mount failed.")
print("Drive mounted successfully.")


# 2. GPU Verification

print("\nChecking GPU availability...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device detected: {device}")


# 3. Create Clean Folder Structure

base_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
raw_data_path = os.path.join(base_path, 'raw_data')

real_path = os.path.join(raw_data_path, 'real')
fake_path = os.path.join(raw_data_path, 'fake')

os.makedirs(real_path, exist_ok=True)
os.makedirs(fake_path, exist_ok=True)

print("Folder structure verified.")


# 4. Connect to OpenFake Dataset

print("\nConnecting to OpenFake dataset stream...")
dataset_name = "ComplexDataLab/OpenFake"

try:
    openfake = load_dataset(dataset_name, split='train', streaming=True)
    print("Dataset connection successful.")
except Exception as e:
    raise RuntimeError(f"Dataset loading failed: {e}")


# 5. Dataset Schema & Label Sanity Check

print("\nPerforming schema + label sanity check...")

sample_preview = None
for i, sample in enumerate(openfake):
    sample_preview = sample
    break

if sample_preview is None:
    raise RuntimeError("Dataset appears empty.")

print("Sample keys:", sample_preview.keys())

if 'label' not in sample_preview or 'image' not in sample_preview:
    raise RuntimeError("Dataset schema unexpected. Required fields missing.")

print("Label of first sample:", sample_preview['label'])

# Reset stream after preview
openfake = load_dataset(dataset_name, split='train', streaming=True)


# 6. Download 25 Real + 25 Fake Images

real_count = 0
fake_count = 0
save_errors = 0
max_iterations = 10000
iteration_counter = 0

print("\nStarting micro-download (25 Real + 25 Fake)...")

for item in openfake:
    iteration_counter += 1

    if iteration_counter > max_iterations:
        raise RuntimeError("Iteration cap exceeded. Possible label imbalance or streaming issue.")

    try:
        label = item['label']
        image = item['image']

        # THE FIX - Looking for strings instead of integers
        if label not in ['real', 'fake']:
            continue  # skip unexpected labels safely

        if label == 'real' and real_count < 25:
            image.save(os.path.join(real_path, f"test_openfake_real_{real_count}.jpg"))
            real_count += 1

        elif label == 'fake' and fake_count < 25:
            image.save(os.path.join(fake_path, f"test_openfake_fake_{fake_count}.jpg"))
            fake_count += 1

        if real_count == 25 and fake_count == 25:
            break

    except Exception as e:
        save_errors += 1
        print(f"Error saving image: {e}")


# 7. Final Integrity Check

print("\nDownload Summary:")
print(f"Iterations used: {iteration_counter}")
print(f"Real images saved: {real_count}")
print(f"Fake images saved: {fake_count}")
print(f"Save errors: {save_errors}")

if real_count != 25 or fake_count != 25:
    raise RuntimeError("Image count mismatch. Micro-test failed.")

print("\n✅ Micro-test completed successfully and verified.")

Requesting Google Drive access...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully.

Checking GPU availability...
Device detected: cpu
Folder structure verified.

Connecting to OpenFake dataset stream...


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Dataset connection successful.

Performing schema + label sanity check...
Sample keys: dict_keys(['image', 'prompt', 'label', 'model', 'type', 'release_date'])
Label of first sample: real


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]


Starting micro-download (25 Real + 25 Fake)...

Download Summary:
Iterations used: 61
Real images saved: 25
Fake images saved: 25
Save errors: 0

✅ Micro-test completed successfully and verified.
